In [10]:
import pandas as pd
import numpy as np
import time
from matplotlib import pyplot as plt
import skimage as skimage
from datetime import date

In [2]:
import torch
import torchvision

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import torch.nn as nn
import torch as t

from torch.utils.data import DataLoader
from torchvision.transforms import functional as F

In [4]:
from data_utils.wider_dataset import *
from data_utils.wider_eval import *
from data_utils.data_read import *
from model.model_utils import *
from model.fasterrcnn import *
from data_utils.utils import *


In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
wider_img_list, wider_bboxes = wider_read(150)

In [11]:
train_dataset = WiderDataset(wider_img_list[:140], wider_bboxes[:140])
test_dataset = WiderDataset(wider_img_list[10:], wider_bboxes[10:])
train_loader = DataLoader(train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=6,
    collate_fn=collate_fn
)
test_loader = DataLoader(test_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=6,
    collate_fn=collate_fn
)

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 2  # 1 class (face) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [6]:
backbone = torchvision.models.detection.backbone_utils.resnet_fpn_backbone('resnet18', True)
model.backbone = backbone

In [7]:

model = load_Faster_RCNN(backbone='resnet18')

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [14]:
model.load_state_dict(torch.load('./saved_models/fasterrcnn_resnet18_fpn3.pth'))

<All keys matched successfully>

In [15]:
model_eval(model, test_loader)

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/visionad/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/anaconda3/envs/visionad/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/anaconda3/envs/visionad/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/susanthdasari/OneDrive - Georgia State University/Projects/Adver-Face-Detection/data_utils/wider_dataset.py", line 51, in __getitem__
    im = totensor(im)
  File "/Users/susanthdasari/OneDrive - Georgia State University/Projects/Adver-Face-Detection/data_utils/arraytools.py", line 20, in totensor
    if isinstance(data, 